<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Introduction to </b> <span style="font-weight:bold; color:green">Amazon Web Services (AWS)</span></div><hr>
<div style="text-align:right;">Sergei Yu. Papulin <span style="font-style: italic;font-weight: bold;">(papulin_bmstu@mail.ru, papulin_hse@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Content</span>
    <ol>
        <li><a href="#1">AWS CLI Installation</a></li>
        <li><a href="#2">Deploying AWS Instances</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#2a">Virtual Private Cloud</a></li>
                <li><a href="#2b">Subnet</a></li>
                <li><a href="#2c">Gateway</a></li>
                <li><a href="#2d">Route Table</a></li>
                <li><a href="#2e">Security Group</a></li>
                <li><a href="#2f">AWS Instance</a></li>
            </ol>
        </li>
        <li><a href="#3">SSH connection to AWS VM</a></li>
        <li><a href="#4">Data Transfer</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#4a">From Local Computer to AWS VM through SSH Connection</a></li>
                <li><a href="#4b">From Remote Site to AWS VM directly through wget-command</a></li>
            </ol>
        </li>
        <li><a href="#5">Resource Deallocation</a></li>
        <li><a href="#6">References</a></li>
    </ol>
</div>

<p>Launch the cell below to apply a jupyter notebook style</p>

In [1]:
%%html
<link href="css/style.css" rel="stylesheet" type="text/css">

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. AWS CLI Installation</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To content</a></div>
    </div>
</div>

<p><b>Create IAM User</b></p>

<p>Log in to your AWS account -> Services -> IAM -> Users -> Add User. Fill in a name field and check the option "Programmatic access" -> Next -> Attach ... -> AmazonEC2FullAccess -> Create User -> Save Keys for the following use</p>

In [ ]:
Access key ID
Secret access key

<div class="msg-block msg-ref">
  <div class="msg-text-ref"><p>For more information use the link below<br><a href="http://docs.aws.amazon.com/cli/latest/userguide/cli-chap-getting-started.html">Configuring the AWS CLI</a></p></div>
</div>

<p><b>Install AWS CLI</b></p>

<p>On Ubuntu</p>

In [ ]:
sudo pip install awscli

<div class="msg-block msg-ref">
  <div class="msg-text-ref"><p>For more information use the link below<br><a href="http://docs.aws.amazon.com/cli/latest/userguide/awscli-install-linux.html">Installing the AWS Command Line Interface</a></p></div>
</div>

<p><b>Configure AWS CLI</b></p>

In [ ]:
sudo aws configure

<p>Fill out the fields below in your terminal</p>

In [ ]:
AWS Access Key ID
AWS Secret Access
Default region name: eu-west-1
Default output format: json

<p>Print out a list of your instances, i.e. VMs</p>

In [ ]:
!sudo aws ec2 describe-instances

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Deploying AWS Instances </div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To content</a></div>
    </div>
</div>

<a name="2a"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Virtual Private Cloud
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2">Back</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#2b">Next</a>
            </div>
        </div>
    </div>
</div>

<p>Create VPC</p>

In [ ]:
%%bash
aws ec2 create-vpc \
    --cidr-block "10.0.1.0/24" \
    --instance-tenancy "default"

<div class="msg-block msg-info">
  <div class="msg-text-info"><p>When you create a VPC, a DHCP, Security Group and Route Table are created automatically</p></div>
</div>

<p>Assign the VPC Id to a python variable</p>

In [ ]:
vpc_id = "vpc-xxxxx"

<p>Attach a name to the VPC</p>

In [ ]:
%%bash -s "$vpc_id"
aws ec2 create-tags \
    --resources $1 \
    --tags Key="Name",Value="BigData_VPC"

<p>Show all VPC. Check a name of the created VPC</p>

In [ ]:
!aws ec2 describe-vpcs --filters "Name=tag:Name,Values=BigData_VPC"

<a name="2b"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Subnet
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2a">Back</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#2c">Next</a>
            </div>
        </div>
    </div>
</div>

<p>Create a subnet</p>

In [ ]:
%%bash -s "$vpc_id"
aws ec2 create-subnet \
    --vpc-id $1 \
    --cidr-block "10.0.1.0/28"

<p>Assign the Subset Id to a python variable</p>

In [ ]:
subnet_id = "subnet-xxxxx"

<p>Attach a name to the subnet</p>

In [ ]:
%%bash -s "$subnet_id"
aws ec2 create-tags \
    --resources $1 \
    --tags Key="Name",Value="BigData_Subnet"

<p>Show all subnets of the VPC. Check a name of the created VPC</p>

In [ ]:
%%bash -s "$subnet_id"
aws ec2 describe-subnets --filters "Name=subnet-id,Values=$1"

<p>or use a tag name for filtering</p>

In [ ]:
!aws ec2 describe-subnets --filters "Name=tag:Name,Values=BigData_Subnet"

<a name="2c"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            c. Gateway
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2b">Back</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#2d">Next</a>
            </div>
        </div>
    </div>
</div>

<p>Create a gateway</p>

In [ ]:
!aws ec2 create-internet-gateway

<p>Assign the Gateway Id to a python variable</p>

In [ ]:
gateway_id = "igw-xxxxx"

<p>Set a tag to refer to the Gateway</p>

In [ ]:
%%bash -s "$gateway_id"
aws ec2 create-tags \
    --resources $1 \
    --tags Key="Name",Value="BigData_Gateway"

<p>Print out your existing gateways</p>

In [ ]:
!aws ec2 describe-internet-gateways --filters "Name=tag:Name,Values=BigData_Gateway"

<p>Attach the internet gateway to the VPC</p>

In [ ]:
%%bash -s "$vpc_id" "$gateway_id"
aws ec2 attach-internet-gateway \
    --vpc-id $1 \
    --internet-gateway-id $2

<a name="2d"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            d. Route Table
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2c">Back</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#2e">Next</a>
            </div>
        </div>
    </div>
</div>

<p>Create a route table</p>

<div class="msg-block msg-info">
  <div class="msg-text-info"><p>To create a route table through the AWS CLI, look at the link below<br><a href="http://docs.aws.amazon.com/cli/latest/reference/ec2/create-route-table.html">create-route-table</a></p>
  <p>Example</p>
  <p class="code-block code-font">aws ec2 <span class="code-key">create-route-table</span> --vpc-id "vpc-xxxxx"</p>
  </div>
</div>

<p>Show the Route Table that was built together with the VPC by default</p>

In [ ]:
%%bash -s "$vpc_id" 
aws ec2 describe-route-tables --filters "Name=vpc-id,Values=$1"

<p>Assign the Route Table Id to a python variable</p>

In [ ]:
rtb_id = "rtb-xxxxx"

<p>Create a route to the gateway</p>

In [ ]:
%%bash -s "$rtb_id" "$gateway_id"
aws ec2 create-route \
    --route-table-id $1 \
    --destination-cidr-block "0.0.0.0/0" \
    --gateway-id $2

<p>Set a tag to refer to the Route Table</p>

In [ ]:
%%bash -s "$rtb_id"
aws ec2 create-tags \
    --resources $1 \
    --tags Key="Name",Value="BigData_RouteTable"

<p>Show your Route Tables with all created routes</p>

In [ ]:
!aws ec2 describe-route-tables --filters "Name=tag:Name,Values=BigData_RouteTable"

<a name="2e"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            e. Security Group
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2d">Back</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#2f">Next</a>
            </div>
        </div>
    </div>
</div>

<p>Create a security group</p>

<div class="msg-block msg-info">
  <div class="msg-text-info"><p>To create a security group through the AWS CLI, look at the link below<br><a href="http://docs.aws.amazon.com/cli/latest/reference/ec2/create-security-group.html">create-security-group
</a></p>
  <p>Example</p>
  <p class="code-block code-font">aws ec2 <span class="code-key">create-security-group</span> \ <br>
  --group-name "BigData_SecurityGroup" \<br>
  --description "Security group for SSH access" \<br>
  --vpc-id "vpc-xxxxx"></p>
  </div>
</div>

<p>Show the Security Group that was built together with the VPC by default</p>

In [ ]:
%%bash -s "$vpc_id"
aws ec2 describe-security-groups --filters "Name=vpc-id,Values=$1"

<p>Assign the Security Group Id to a python variable</p>

In [ ]:
secgroup_id = "sg-xxxxx"

<p>Set a tag to refer to the Security Group</p>

In [ ]:
%%bash -s "$secgroup_id"
aws ec2 create-tags \
    --resources $1 \
    --tags Key="Name",Value="BigData_SecurityGroup"

<p><b>Modify the security group created by default</b></p>

Allow the SSH access

In [ ]:
%%bash -s "$secgroup_id"
aws ec2 authorize-security-group-ingress \
    --group-id $1 \
    --protocol tcp \
    --port 22 \
    --cidr "0.0.0.0/0"

<p>Remove an egress rule</p>

In [ ]:
%%bash -s "$secgroup_id"
aws ec2 revoke-security-group-egress \
    --group-id $1 \
    --protocol all \
    --cidr "0.0.0.0/0"

<p>Allow all traffic from instances that belong to the Security Group</p>

In [ ]:
%%bash -s "$secgroup_id"
aws ec2 authorize-security-group-egress \
    --group-id $1 \
    --protocol all \
    --cidr "0.0.0.0/0"

<p>Show the Security Group after modification</p>

In [ ]:
%%bash -s "$secgroup_id"
aws ec2 describe-security-groups --filter Name="group-id",Values=$1

<a name="2f"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            f. AWS Instance
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2e">Back</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#3">Next</a>
            </div>
        </div>
    </div>
</div>

<p><b>Option 1.</b> Create a key pair</p>

In [ ]:
!aws ec2 create-key-pair --key-name "BigData_Keys"

<p><b>Option 2.</b> Import an existing key</p>

In [ ]:
%%bash
aws ec2 import-key-pair \
    --key-name "BigData_Keys" \
    --public-key-material "ssh-rsa ....."

<div class="msg-block msg-ref">
  <div class="msg-text-ref"><p>For more information use the link below<br><a href="http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html">Amazon EC2 Key Pairs</a></p></div>
</div>

<p>Launch an AWS instance</p>

In [ ]:
%%bash -s "$subnet_id" "$secgroup_id"
aws ec2 run-instances \
    --image-id "ami-2944b450" \
    --count 1 \
    --instance-type "t2.micro" \
    --key-name "BigData_Keys" \
    --subnet-id $1 \
    --security-group-ids $2 \
    --instance-initiated-shutdown-behavior "stop" \
    --private-ip-address "10.0.1.5" \
    --associate-public-ip-address \
    --tag-specifications "ResourceType=instance,Tags=[{Key=Name,Value=VM1}]"

<div class="msg-block msg-info">
  <div class="msg-text-info"><p>To know more about available instance types, look at here - <a href="http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/instance-types.html">Instance Types</a></p></div>
</div>

<div class="msg-block msg-info">
  <div class="msg-text-info"><p>To find other Ubuntu AMIs that can be used in the AWS, click on the following link - <a href="https://cloud-images.ubuntu.com/locator/ec2/">Amazon EC2 AMI Locator</a></p></div>
</div>

<p>Assign the VM1 Id to a python variable</p>

In [ ]:
vm1_id = "i-xxxxx"

<p>Use a custom Block Volume parameters to an instance using a block device mapping</p>

In [ ]:
%%bash -s "$subnet_id" "$secgroup_id"
aws ec2 run-instances \
    --image-id "ami-2944b450" \
    --count 1 \
    --instance-type "t2.micro" \
    --key-name "BigData_Keys" \
    --subnet-id $1 \
    --security-group-ids $2 \
    --instance-initiated-shutdown-behavior "stop" \
    --private-ip-address "10.0.1.6" \
    --associate-public-ip-address \
    --block-device-mappings '[{"DeviceName": "/dev/sda1", "Ebs": { "DeleteOnTermination": false, "VolumeSize": 16, "VolumeType": "gp2" }}]' \
    --tag-specifications "ResourceType=instance,Tags=[{Key=Name,Value=VM2}]"

<p>Assign the VM2 Id to a python variable</p>

In [ ]:
vm2_id = "i-xxxxx"

<p>Show volumes used by an instance</p>

In [ ]:
%%bash -s "$vm2_id"
aws ec2 describe-volumes --filter Name="attachment.instance-id",Values=$1

Assign the Volume Id to a python variable

In [ ]:
vol_id = "vol-xxxxx"

<div class="msg-block msg-ref">
  <div class="msg-text-ref"><p>For more information use the link below<br><a href="http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/block-device-mapping-concepts.html">Block Device Mapping</a></p></div>
</div>

<p>Show a public ip of the created instance (VM)</p>

In [ ]:
%%bash
aws ec2 describe-instances \
    --query "Reservations[?ReservationId=='r-xxxxx'].Instances[*].PublicIpAddress" \
    --output text

In [ ]:
# TODO: Change to use an instance ID

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. SSH connection to AWS VM</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To content</a></div>
    </div>
</div>

<p>Restrict permission to your private key file (.pem)</p>

In [ ]:
sudo chmod 400 "/PATH/TO/bigdata_keys.pem"

<p>Connect to the remote AWS instance/VM through SSH</p>

In [ ]:
sudo ssh -i "/PATH/TO/bigdata_keys.pem" ubuntu@INSTANCE_PUBLIC_IP

<p>Print out a list of directories of the VM</p>

In [ ]:
ls /

<div class="msg-block msg-ref">
  <div class="msg-text-ref"><p>For more information use the link below<br><a href="http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AccessingInstancesLinux.html">Connecting to Your Linux Instance Using SSH</a></p></div>
</div>

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Data Transfer</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To content</a></div>
    </div>
</div>

<a name="4a"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. From Local Computer to AWS VM through SSH Connection
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#4">Back</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#4b">Next</a>
            </div>
        </div>
    </div>
</div>

<p><b><i>Remote AWS VM</i></b></p>

<p>Connect to the remote VM through SSH and create a directory</p>

In [ ]:
mkdir /home/ubuntu/test_scp_dir

<p>Check that the directory was created</p>

In [ ]:
ls /home/ubuntu/

<p><b><i>Local Node</i></b></p>

<p>Copy a local file to the remote AWS VM</p>

In [ ]:
scp -i "/PATH/TO/bigdata_keys.pem" "/PATH/TO/ANY/FILE" ubuntu@INSTANCE_PUBLIC_IP:"/home/ubuntu/test_scp_dir"

<p><b><i>Remote AWS VM</i></b></p>

<p>Verify that the file appears in the directory in the Remote AWS VM</p>

In [ ]:
ls /home/ubuntu/test_scp_dir

<div class="msg-block msg-ref">
  <div class="msg-text-ref"><p>For more information use the link below<br><a href="http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AccessingInstancesLinux.html">Connecting to Your Linux Instance Using SSH</a></p></div>
</div>

<a name="4b"></a>
<div style="display:table; width:100%">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. From Remote Site to AWS VM directly through wget-command
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style="display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#4a">Back</a>
            </div>
            <div style="display:table-cell; width:10%; text-align:center;">
                <a href="#5">Next</a>
            </div>
        </div>
    </div>
</div>

<p><b><i>Remote AWS VM</i></b></p>

<p>Connect to the remote VM through SSH and create a directory</p>

In [ ]:
mkdir /home/ubuntu/test_wget_dir

</p>Use the wget-command to download a file using Internet protocols</p>

In [ ]:
wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip -P "/home/ubuntu/test_wget_dir"

<p>Check that the file was downloaded</p>

In [ ]:
ls /home/ubuntu/test_wget_dir

<a name="5"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">5. Resource Deallocation</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To content</a></div>
    </div>
</div>

<p>Main commands that change a state of AWS instances</p>

<p><i>Stop</i></p>

In [ ]:
%%bash -s "$vm1_id" "$vm2_id" 
aws ec2 stop-instances --instance-ids $1 $2

<div class="msg-block msg-warning">
  <div class="msg-text-warn"><p>Transition between instance states takes some time. You can see an error launching the next command if the transition "running-stopped" is not completed</p></div>
</div>

<p><i>Start</i></p>

In [ ]:
%%bash -s "$vm1_id" "$vm2_id" 
aws ec2 start-instances --instance-ids $1 $2

<p><i>Reboot</i></p>

In [ ]:
%%bash -s "$vm1_id" "$vm2_id" 
aws ec2 reboot-instances --instance-ids $1 $2

<p><i>Terminate</i></p>

In [ ]:
%%bash -s "$vm1_id" "$vm2_id" 
aws ec2 terminate-instances --instance-ids $1 $2

<div class="msg-block msg-ref">
  <div class="msg-text-ref"><p>For more information use the link below<br><a href="http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-instance-lifecycle.html">Instance Lifecycle</a></p></div>
</div>

<p>Delete a EBS Volume</p>

In [ ]:
!aws ec2 delete-volume --volume-id "$vol_id"

<p>Delete a VPC through the AWS console together with your security group, route table, gateway, subnet</p>

<a name="6"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">6. References</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">To content</a></div>
    </div>
</div>

<a href="http://docs.aws.amazon.com/cli/latest/userguide/awscli-install-linux.html">Installing the AWS Command Line Interface</a><br>
<a href="http://docs.aws.amazon.com/cli/latest/reference/ec2/index.html#cli-aws-ec2">AWS CLI EC2</a><br>
<a href="http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-instance-addressing.html">Amazon EC2 Instance IP Addressing</a><br>